---

# Lib

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import re

from preprocessing.clean import vn_text_clean
from preprocessing.tokenize import vn_word_tokenize

[nltk_data] Downloading package punkt to C:\Users\DANG
[nltk_data]     CUONG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---

# Read file

In [2]:
df = pd.read_csv("../data/raw/train.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [3]:
df.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [4]:
df.drop(columns=['index', 'n_star', 'date_time'], inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7786 entries, 0 to 7785
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  7786 non-null   object
 1   label    7786 non-null   object
dtypes: object(2)
memory usage: 121.8+ KB


# Clean text

In [6]:
df["comment"] = df["comment"].apply(lambda x: vn_text_clean(x))

In [7]:
df.head()

,comment,label
0,mới mua máy này tại thegioididong thốt nốt cảm...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,pin kém còn lại miễn chê mua tình trạng pin cò...,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,{FEATURES#Negative};
3,mọi người cập nhật phần mềm lại nó sẽ bớt tốn ...,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,mới mua sài được tháng thấy pin rất trâu sài b...,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [8]:
df["comment"] = df["comment"].apply(lambda x: vn_word_tokenize(x, method="underthesea"))

In [9]:
df.head()

,comment,label
0,mới mua máy này tại thegioididong thốt nốt cảm...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,pin kém còn lại miễn_chê mua tình_trạng pin cò...,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,sao lúc gọi điện_thoại màn_hình bị chấm nhỏ nh...,{FEATURES#Negative};
3,mọi người cập_nhật phần_mềm lại nó sẽ bớt tốn ...,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,mới mua sài được tháng thấy pin rất trâu sài b...,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# Save file

In [ ]:
df.to_csv("../data/processed/train_preprocessed.csv", index=False)